# Load packages

In [1]:
import time
import os
import sys
import numpy as np
import tensorflow as tf
from six.moves import xrange
import matplotlib.pyplot as plt

import transformation
import load_dataset2
import config

conf,_ = config.get_config()

c:\users\mmnet\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Namespace(agl_dim=2, batch_size=256, channel=3, data_dir='../../dataset/', dataset='dirl_48x64_example', early_stop=16, easy_mode=True, ef_dim=12, encoded_agl_dim=16, epochs=500, eye='L', height=48, load_weights=False, loss_combination='l2sc', lr=0.001, tar_model='flx', tb_dir='TFboard/', train_dir='training_inputs/', valid_dir='valid_inputs/', weight_dir='pt_ckpt/', width=64)


In [2]:
if conf.tar_model == "flx":
    import flx as model
else:
    sys.exit("Sorry, Wrong Model!")
# tar_model='deepwarp'
if conf.loss_combination == 'l2sc' or conf.loss_combination == 'l2s' or conf.loss_combination == 'l2':
    print ('Loss Type is', conf.loss_combination)
else:
    sys.exit("Sorry, Wrong Loss Combination!")

Loss Type is l2sc


# Parameters, folders, and #GPUs

In [3]:
tf.set_random_seed(711129)
'''setting'''
gpus = [0,1] # Here I set CUDA to only see one GPU
os.environ['CUDA_VISIBLE_DEVICES']=','.join([str(i) for i in gpus])
n_gpus = len(gpus) # number of GPUs to use

# Functions from TF (Do not change)

In [4]:
def average_gradients(tower_grads):
    """Calculate the average gradient for each shared variable across all towers.
    Note that this function provides a synchronization point across all towers.
    Args:
      tower_grads: List of lists of (gradient, variable) tuples. The outer list
        is over individual gradients. The inner list is over the gradient
        calculation for each tower.
    Returns:
       List of pairs of (gradient, variable) where the gradient has been averaged
       across all towers.
    """
    average_grads = []
    
    for grad_and_vars in zip(*tower_grads):
      # Note that each grad_and_vars looks like the following:
      #   ((grad0_gpu0, var0_gpu0), ... , (grad0_gpuN, var0_gpuN))
        grads = []
        
        for g, _ in grad_and_vars:
            # Add 0 dimension to the gradients to represent the tower.
            expanded_g = tf.expand_dims(g, 0)
            # Append on a 'tower' dimension which we will average over below.
            grads.append(expanded_g)

        # Average over the 'tower' dimension.
        grad = tf.concat(axis=0, values=grads)
        grad = tf.reduce_mean(grad, 0)

        # Keep in mind that the Variables are redundant because they are shared
        # across towers. So .. we will just return the first tower's pointer to
        # the Variable.
        v = grad_and_vars[0][1]
        grad_and_var = (grad, v)
        average_grads.append(grad_and_var)
    return average_grads

# Tower Loss (define your model here)

In [5]:
def tower_loss(scope, input_img, input_fp, input_ang, img_, phase_train, eye_mask, loss_combination):
    # model
    img_pred, flow, lcm_wgts,_ = model.inference(input_img, input_fp, input_ang, phase_train, conf)
    
    # calcuate loss
    with tf.name_scope('loss'):
        _, l2_loss = model.loss(img_pred, img_, eye_mask, input_img, flow, lcm_wgts, loss_combination)
        
    losses = tf.get_collection('losses', scope)
    # Calculate the total loss for the current tower.
    total_loss = tf.add_n(losses, name='total_loss')
    return total_loss, img_pred, flow, lcm_wgts, l2_loss

# Determine Model

In [6]:
TOWER_NAME = 'tower'
with tf.Graph().as_default(), tf.device('/cpu:0'):
    # define step counter
    global_step = tf.get_variable('global_step', [], initializer=tf.constant_initializer(0), trainable=False)
    # define lr and bn control
    with tf.name_scope('model_control'):
        phase_train = tf.placeholder(tf.bool, name='phase_train')
        learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    # define inputs
    with tf.name_scope('inputs'):
        input_img = tf.placeholder(tf.float32, [None, conf.height, conf.width, conf.channel], name='input_img') # [None, 41, 51, 3]
        input_fp = tf.placeholder(tf.float32, [None, conf.height, conf.width,conf.ef_dim], name='input_fp') # [None, 41, 51, 14]
        input_ang = tf.placeholder(tf.float32, [None, conf.agl_dim], name='input_ang') ## [None, 2]        
        img_ = tf.placeholder(tf.float32, [None, conf.height, conf.width, conf.channel], name ='Ground_Truth') # [None, 41, 51, 3]
        input_eye_msk = tf.placeholder(tf.float32, [None, conf.height, conf.width], name='input_eye_msk') # [None, 41, 51]

    # define optimizer
    opt = tf.train.AdamOptimizer(learning_rate, beta1=0.5)
    
    # define Tower
    tower_grads = []
    tower_synImgs = []
    tower_CM = []
    tower_Tloss=[]
    tower_L2loss=[]
    with tf.variable_scope('gen'):
        for i in xrange(n_gpus):
            with tf.device('/gpu:%d' % i):
                with tf.name_scope('%s_%d' % (TOWER_NAME, i)) as scope:
                    print('%s_%d' % (TOWER_NAME, i))
                    Tloss, synImgs, _, ss_CM, L2loss = tower_loss(scope,
                                                                input_img[i*conf.batch_size:(i+1)*conf.batch_size],
                                                                input_fp[i*conf.batch_size:(i+1)*conf.batch_size],
                                                                input_ang[i*conf.batch_size:(i+1)*conf.batch_size],
                                                                img_[i*conf.batch_size:(i+1)*conf.batch_size],                                 
                                                                phase_train,
                                                                input_eye_msk[i*conf.batch_size:(i+1)*conf.batch_size],
                                                                conf.loss_combination)
                    # Reuse variables for the next tower.
                    tf.get_variable_scope().reuse_variables()
                    
                    # get generator parameter list
                    gen_vars = [var for var in tf.trainable_variables() if var.name.startswith("gen")] #; print(gen_vars)

                    # Calculate the gradients for the batch of data on this tower.
                    grads = opt.compute_gradients(Tloss, var_list=gen_vars)
                    # Keep track of the gradients across all towers.
                    tower_grads.append(grads)
                    tower_synImgs.append(synImgs)
                    tower_CM.append(ss_CM)
                    tower_Tloss.append(Tloss)
                    tower_L2loss.append(L2loss)

        gen_grads = average_gradients(tower_grads)
        pred_synImgs = tf.reshape(tower_synImgs, shape = [n_gpus*conf.batch_size, conf.height, conf.width,3])
        pred_CM = tf.reshape(tower_CM, shape = [n_gpus*conf.batch_size, conf.height, conf.width,2])
        pred_Tloss = tf.reshape(tower_Tloss, shape = [n_gpus])
        pred_L2loss = tf.reshape(tower_L2loss, shape = [n_gpus])

    # apply gradient to weights
    apply_gradient_op = opt.apply_gradients(gen_grads, global_step=global_step)

    # weight init
    init = tf.global_variables_initializer()
    
    # Create a saver
    saver = tf.train.Saver(tf.global_variables())
    saver2 = tf.train.Saver(tf.global_variables())

    extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

    # define tf.sess
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False))

    # define graph writer
    writer = tf.summary.FileWriter(conf.tb_dir, graph = sess.graph)
    
    total_parameters = 0
    for variable in tf.trainable_variables():
        # shape is an array of tf.Dimension
        shape = variable.get_shape()

        variable_parameters = 1
        for dim in shape:
            variable_parameters *= dim.value

        total_parameters += variable_parameters
    print('total_parameters', total_parameters)

tower_0
tower_1
total_parameters 249328


# Load training & validation datasets

In [7]:
train_dicts = load_dataset2.get_dict(conf, 'training_inputs/')
valid_dicts = load_dataset2.get_dict(conf, 'valid_inputs/')

Pose dirs ['0']
pose 0
Dirs ['0000' '0001' '0002']
../../dataset/dirl_48x64_example\training_inputs/0\0000
../../dataset/dirl_48x64_example\training_inputs/0\0001
../../dataset/dirl_48x64_example\training_inputs/0\0002
Pose dirs ['0']
pose 0
Dirs ['0008']
../../dataset/dirl_48x64_example\valid_inputs/0\0008


# Create data iterators

In [8]:
train_iter_ = load_dataset2.get_easy_hard_iter(train_dicts, int(n_gpus*conf.batch_size/4))
valid_iter_ = load_dataset2.get_easy_hard_iter(valid_dicts, int(n_gpus*conf.batch_size/4))
# [0] easy iter, [1] hard iter, [2] #pair of easy iter, [3] #pair of hard iter

E 3219; H 26186; ALL 29405
E 1010; H 8990; ALL 10000


In [9]:
def print_result(ckpt_dir, valid_batch, valid_synImg, valid_CM, n_epoch):
    fig_per_loop = 5
    img_per_row = 5
    for i in range(fig_per_loop):
        plt.subplot(fig_per_loop, img_per_row, fig_per_loop*i+1)
        plt.imshow(np.asarray(valid_batch['imgs_ori'])[fig_per_loop + i,...])
        plt.axis('off')

        plt.subplot(fig_per_loop, img_per_row, fig_per_loop*i+2)
        plt.imshow(np.asarray(valid_batch['imgs__ori'])[fig_per_loop + i,...])
        plt.axis('off')

        plt.subplot(fig_per_loop, img_per_row, fig_per_loop*i+3)
        plt.imshow(np.asarray(valid_synImg)[fig_per_loop + i,...])
        plt.axis('off')

        dif_img = np.absolute(np.asarray(valid_synImg)[fig_per_loop + i,...]-np.asarray(valid_batch['imgs__ori'])[fig_per_loop + i,...])
        dif_img_gray = np.sum(dif_img,axis =2)
        plt.subplot(fig_per_loop, img_per_row, fig_per_loop*i+4)
        plt.imshow(dif_img_gray, cmap='gray')
        plt.axis('off')

        plt.subplot(fig_per_loop, img_per_row, fig_per_loop*i+5)
        plt.imshow(np.asarray(valid_CM)[fig_per_loop + i,...,1], cmap='gray')
        plt.axis('off')

    plt.savefig(ckpt_dir+'imgs/'+'/imgs-'+str(n_epoch)+'.png')
#     plt.show()
    plt.close('all')

In [10]:
def train(conf, ckpt_dir, train_iter_, valid_iter_, easy_mode=True, load_weights=True):
    # load modeling if we want
    if (load_weights):
        ckpt = tf.train.get_checkpoint_state(conf.weight_dir+conf.tar_model+'_'+conf.loss_combination+'/'+str(conf.ef_dim)+'/'+conf.eye+'/')
        if ckpt and ckpt.model_checkpoint_path:
            if (easy_mode):
                saver.restore(sess, ckpt.model_checkpoint_path)
            else:
                saver2.restore(sess, ckpt.model_checkpoint_path)
            
            print('Loading sucessfully')
        else:
            print('No checkpoint file found')
            raise
    else:
        sess.run(init)
        
    if not os.path.exists(ckpt_dir+'imgs/'):
        os.makedirs(ckpt_dir+'imgs/') 
    
    # define init. constants for training
    stop_counter = 0
    min_Tloss = 9999
    min_L2loss = 9999
    learning_decay_count = 0
    cur_lr = conf.lr
    
    # prepare monitoring .csv file
    fout= open(ckpt_dir+conf.loss_combination+'_'+conf.dataset+'_'+conf.eye+'.csv', 'w')
    if not os.path.exists(ckpt_dir+'imgs/'):
        os.makedirs(ckpt_dir+'imgs/')

    fout.write(str('step,train_loss,valid_loss,img_diff,hard_valid_loss,hard_img_diff \n'))         

    if (easy_mode):
        step_per_epoch = int(train_iter_[2]/(n_gpus*conf.batch_size))
    else:
        step_per_epoch = int(4*train_iter_[3]/(n_gpus*conf.batch_size))
        
    for n_epoch in range(conf.epochs):
        epoch_start_time = time.time()
        print('------%d/%d------' % (n_epoch, conf.epochs))
        steps = 0

        # training step
        for s in range(step_per_epoch):
            batch_start_time = time.time()            
            steps = (n_epoch*step_per_epoch + s)

            # ------- start validation -------
            if (steps % int(step_per_epoch/2) == 0):
                # for a validation epoch
                valid_easy_Tlosses = []
                valid_easy_L2losses = []
                valid_hard_Tlosses = []
                valid_hard_L2losses = []

                # hard part
                for s in range(int(valid_iter_[3]/(n_gpus*conf.batch_size))):
                    # get 4 sub_batch from hard iter
                    temp_batch = next(valid_iter_[1])
                    temp_batch = load_dataset2.merge_batches(temp_batch,next(valid_iter_[1]))
                    temp_batch = load_dataset2.merge_batches(temp_batch,next(valid_iter_[1]))
                    temp_batch = load_dataset2.merge_batches(temp_batch,next(valid_iter_[1]))
                    valid_batch = load_dataset2.shuffle_data_batch(temp_batch)

                    valid_synImg, valid_CM, valid_Tloss, valid_L2loss = sess.run([pred_synImgs, pred_CM, pred_Tloss, pred_L2loss],
                                                                                  feed_dict={                                                                  
                                                                                      input_img: valid_batch['imgs_ori'],
                                                                                      input_fp: valid_batch['fp'][:,:,:,0:conf.ef_dim],
                                                                                      input_ang: valid_batch['tar_agl']-valid_batch['sur_agl'],
                                                                                      img_: valid_batch['imgs__ori'],
                                                                                      learning_rate: cur_lr,
                                                                                      phase_train: False,
                                                                                      input_eye_msk: valid_batch['msk_eye']
                                                                                  })
                    valid_hard_Tlosses.append(valid_Tloss)
                    valid_hard_L2losses.append(valid_L2loss)

                # easy part
                for s in range(int(valid_iter_[2]/(n_gpus*conf.batch_size))):
                    # get 4 sub_batch from easy iter
                    temp_batch = next(valid_iter_[0])
                    temp_batch = load_dataset2.merge_batches(temp_batch,next(valid_iter_[0]))
                    temp_batch = load_dataset2.merge_batches(temp_batch,next(valid_iter_[0]))
                    temp_batch = load_dataset2.merge_batches(temp_batch,next(valid_iter_[0]))
                    valid_batch = load_dataset2.shuffle_data_batch(temp_batch)

                    valid_synImg, valid_CM, valid_Tloss, valid_L2loss = sess.run([pred_synImgs, pred_CM, pred_Tloss, pred_L2loss],
                                                                                  feed_dict={                                                                  
                                                                                      input_img: valid_batch['imgs_ori'],
                                                                                      input_fp: valid_batch['fp'][:,:,:,0:conf.ef_dim],
                                                                                      input_ang: valid_batch['tar_agl']-valid_batch['sur_agl'],
                                                                                      img_: valid_batch['imgs__ori'],
                                                                                      learning_rate: cur_lr,
                                                                                      phase_train: False,
                                                                                      input_eye_msk: valid_batch['msk_eye']
                                                                                  })
                    valid_easy_Tlosses.append(valid_Tloss)
                    valid_easy_L2losses.append(valid_L2loss)

                VeT = np.mean(valid_easy_Tlosses)
                VeL2 = np.mean(valid_easy_L2losses)
                VhT = np.mean(valid_hard_Tlosses)
                VhL2 = np.mean(valid_hard_L2losses)

                fout.write(str('%d,NA,%.4f,%.4f,%.4f,%.4f\n' % (steps, VeT, VeL2, VhT, VhL2)))
                fout.flush()

                # show some validation synthesized images
                print_result(ckpt_dir, valid_batch, valid_synImg, valid_CM,n_epoch)
               
                # early stop based on the validation results
                if((VeT < min_Tloss) or (VeL2 < min_L2loss)):
                    if (VeT < min_Tloss):
                        min_Tloss = VeT
                    if (VeL2 < min_L2loss):
                        min_L2loss = VeL2
                    stop_counter = 0
                    print('-- Step %d, [V*] eT=%.1f, eL2=%.1f; hT=%.1f, hL2=%.1f' % (steps, VeT, VeL2, VhT, VhL2))

                    if(easy_mode):
                        saver.save(sess, ckpt_dir+conf.dataset+'_'+conf.eye+str(n_epoch), global_step)
                    else:
                        saver2.save(sess, ckpt_dir+conf.dataset+'_'+conf.eye+str(n_epoch), global_step)
                else:
                    stop_counter = stop_counter +1
                    print('-- Step %d, [V] eT=%.1f, eL2=%.1f; hT=%.1f, hL2=%.1f' % (steps, VeT, VeL2, VhT, VhL2))
                    if (stop_counter % np.floor(conf.early_stop / 3) == 0):
                        learning_decay_count = learning_decay_count + 1
                        cur_lr = conf.lr * np.power(0.9, learning_decay_count)
                        print('Decreasing lr to %f' % cur_lr)
                        
                    # break step
                    if (stop_counter > conf.early_stop):
                        print('Early stop at step %d' % steps)
                        
                        break

            # ------- start training -------
            # get batch
            temp_batch = next(train_iter_[0])
            temp_batch = load_dataset2.merge_batches(temp_batch,next(train_iter_[0]))
            temp_batch = load_dataset2.merge_batches(temp_batch,next(train_iter_[0]))
            if(easy_mode):
                temp_batch = load_dataset2.merge_batches(temp_batch,next(train_iter_[0]))
            else:
                temp_batch = load_dataset2.merge_batches(temp_batch,next(train_iter_[1]))
            
            train_batch = load_dataset2.shuffle_data_batch(temp_batch)

            _,_,train_Tloss,train_L2loss = sess.run([apply_gradient_op, extra_update_ops, pred_Tloss, pred_L2loss],
                                                  feed_dict= {                                                                  
                                                      input_img: train_batch['imgs_ori'],
                                                      input_fp: train_batch['fp'][:,:,:,0:conf.ef_dim],
                                                      input_ang: train_batch['tar_agl']-train_batch['sur_agl'],
                                                      img_: train_batch['imgs__ori'],
                                                      learning_rate: cur_lr,
                                                      phase_train: True,
                                                      input_eye_msk: train_batch['msk_eye']
                                                  })
            batch_duration = time.time() - batch_start_time

            if (steps % step_per_epoch == 0):                   
                print('-- Step %d, [T] Tloss=%.1f, L2loss=%.1f (%.1fs)' % (steps, np.mean(train_Tloss), np.mean(train_L2loss),  batch_duration))
                fout.write(str('%d,%.4f,NA,%.4f,NA,NA\n' % (steps, np.mean(train_Tloss),np.mean(train_L2loss))))
                fout.flush()

        # Stop if early stop
        if (stop_counter > conf.early_stop):
            break
    fout.close()

# Training process (easy)

In [11]:
# direct to pt_ckpt
ckpt_dir = 'pt_ckpt/'+conf.tar_model+'_'+conf.loss_combination+'/'+str(conf.ef_dim)+'/'+conf.eye+'/'

train(conf, ckpt_dir, train_iter_, valid_iter_, True, False)

print('Finish Easy Training, 228')

------0/500------
-- Step 0, [V*] eT=8679.3, eL2=2103.5; hT=8902.6, hL2=2115.1
-- Step 0, [T] Tloss=6451.1, L2loss=1168.1 (20.5s)
-- Step 3, [V*] eT=5349.9, eL2=1696.5; hT=5746.8, hL2=1901.6
------1/500------
-- Step 6, [V*] eT=3941.1, eL2=1349.5; hT=4328.7, hL2=1542.3
-- Step 6, [T] Tloss=3083.8, L2loss=1131.5 (13.8s)
-- Step 9, [V*] eT=3099.0, eL2=1125.8; hT=3384.0, hL2=1263.5
------2/500------
-- Step 12, [V*] eT=2772.8, eL2=1061.0; hT=3024.2, hL2=1186.8
-- Step 12, [T] Tloss=2666.7, L2loss=1088.4 (14.5s)
-- Step 15, [V*] eT=2661.0, eL2=1051.8; hT=2890.0, hL2=1170.0
------3/500------
-- Step 18, [V*] eT=2438.7, eL2=951.3; hT=2697.9, hL2=1105.1
-- Step 18, [T] Tloss=2462.0, L2loss=1014.9 (14.0s)
-- Step 21, [V*] eT=2413.0, eL2=913.6; hT=2635.7, hL2=1054.8
------4/500------
-- Step 24, [V*] eT=2291.4, eL2=885.5; hT=2530.7, hL2=1038.1
-- Step 24, [T] Tloss=2367.5, L2loss=992.4 (14.0s)
-- Step 27, [V*] eT=2276.7, eL2=918.6; hT=2484.0, hL2=1048.7
------5/500------
-- Step 30, [V*] eT=218

-- Step 258, [T] Tloss=654.0, L2loss=425.4 (9.6s)
-- Step 261, [V] eT=719.1, eL2=493.7; hT=884.3, hL2=641.8
Decreasing lr to 0.000900
------44/500------
-- Step 264, [V] eT=723.6, eL2=504.9; hT=914.2, hL2=666.4
-- Step 264, [T] Tloss=632.5, L2loss=410.0 (9.6s)
-- Step 267, [V*] eT=681.8, eL2=466.3; hT=876.9, hL2=641.5
------45/500------
-- Step 270, [V] eT=689.4, eL2=473.8; hT=876.9, hL2=638.0
-- Step 270, [T] Tloss=624.8, L2loss=399.3 (9.6s)
-- Step 273, [V*] eT=659.5, eL2=448.7; hT=844.0, hL2=617.4
------46/500------
-- Step 276, [V] eT=1084.9, eL2=810.3; hT=1259.3, hL2=903.7
-- Step 276, [T] Tloss=630.5, L2loss=436.6 (9.6s)
-- Step 279, [V*] eT=649.5, eL2=444.8; hT=837.0, hL2=610.9
------47/500------
-- Step 282, [V*] eT=639.2, eL2=433.6; hT=814.1, hL2=592.7
-- Step 282, [T] Tloss=604.4, L2loss=384.9 (14.3s)
-- Step 285, [V] eT=656.5, eL2=457.5; hT=866.1, hL2=636.6
------48/500------
-- Step 288, [V] eT=644.4, eL2=440.0; hT=830.9, hL2=599.0
-- Step 288, [T] Tloss=580.3, L2loss=369.4

-- Step 519, [V] eT=439.3, eL2=332.6; hT=652.7, hL2=520.2
------87/500------
-- Step 522, [V*] eT=437.9, eL2=329.6; hT=652.9, hL2=519.1
-- Step 522, [T] Tloss=347.1, L2loss=245.8 (14.5s)
-- Step 525, [V*] eT=427.3, eL2=325.8; hT=635.2, hL2=508.3
------88/500------
-- Step 528, [V] eT=436.0, eL2=331.1; hT=649.8, hL2=518.8
-- Step 528, [T] Tloss=343.9, L2loss=244.3 (9.6s)
-- Step 531, [V*] eT=404.9, eL2=308.2; hT=617.3, hL2=496.6
------89/500------
-- Step 534, [V*] eT=399.0, eL2=298.5; hT=603.6, hL2=481.0
-- Step 534, [T] Tloss=334.8, L2loss=235.1 (14.2s)
-- Step 537, [V] eT=407.7, eL2=311.3; hT=610.9, hL2=490.8
------90/500------
-- Step 540, [V] eT=404.6, eL2=303.9; hT=614.7, hL2=490.3
-- Step 540, [T] Tloss=331.6, L2loss=234.2 (9.7s)
-- Step 543, [V] eT=409.3, eL2=315.2; hT=615.7, hL2=498.3
------91/500------
-- Step 546, [V] eT=399.1, eL2=301.0; hT=606.0, hL2=484.8
-- Step 546, [T] Tloss=329.6, L2loss=236.8 (9.7s)
-- Step 549, [V*] eT=390.4, eL2=299.9; hT=618.1, hL2=500.8
------92/5

-- Step 777, [V] eT=329.7, eL2=266.4; hT=547.4, hL2=456.5
------130/500------
-- Step 780, [V] eT=328.5, eL2=265.0; hT=538.3, hL2=449.4
-- Step 780, [T] Tloss=256.7, L2loss=197.0 (9.7s)
-- Step 783, [V*] eT=322.5, eL2=259.6; hT=536.6, hL2=447.8
------131/500------
-- Step 786, [V] eT=331.3, eL2=267.9; hT=552.6, hL2=461.0
-- Step 786, [T] Tloss=258.4, L2loss=198.5 (9.8s)
-- Step 789, [V] eT=328.8, eL2=265.5; hT=543.1, hL2=452.6
------132/500------
-- Step 792, [V] eT=325.8, eL2=263.2; hT=541.4, hL2=452.8
-- Step 792, [T] Tloss=259.4, L2loss=199.3 (9.8s)
-- Step 795, [V] eT=329.6, eL2=266.8; hT=548.7, hL2=457.9
------133/500------
-- Step 798, [V] eT=326.2, eL2=263.4; hT=552.2, hL2=459.0
Decreasing lr to 0.000229
-- Step 798, [T] Tloss=254.7, L2loss=196.2 (9.9s)
-- Step 801, [V] eT=333.2, eL2=269.9; hT=560.5, hL2=465.3
------134/500------
-- Step 804, [V] eT=329.9, eL2=267.2; hT=559.4, hL2=466.2
-- Step 804, [T] Tloss=257.5, L2loss=198.8 (9.8s)
-- Step 807, [V] eT=328.8, eL2=265.8; hT=55

-- Step 1032, [V] eT=314.6, eL2=257.6; hT=543.1, hL2=457.3
-- Step 1032, [T] Tloss=244.0, L2loss=190.8 (9.6s)
-- Step 1035, [V] eT=316.0, eL2=258.8; hT=544.3, hL2=457.9
Decreasing lr to 0.000072
------173/500------
-- Step 1038, [V] eT=314.5, eL2=258.0; hT=539.8, hL2=454.8
-- Step 1038, [T] Tloss=244.2, L2loss=191.2 (9.8s)
-- Step 1041, [V] eT=313.6, eL2=256.9; hT=538.5, hL2=453.9
------174/500------
-- Step 1044, [V] eT=318.6, eL2=261.6; hT=536.1, hL2=452.4
-- Step 1044, [T] Tloss=242.4, L2loss=189.1 (9.8s)
-- Step 1047, [V] eT=314.7, eL2=258.5; hT=537.9, hL2=454.1
------175/500------
-- Step 1050, [V] eT=315.1, eL2=258.5; hT=532.0, hL2=449.1
Decreasing lr to 0.000065
-- Step 1050, [T] Tloss=243.6, L2loss=190.6 (9.9s)
-- Step 1053, [V] eT=316.3, eL2=259.7; hT=534.8, hL2=452.0
------176/500------
-- Step 1056, [V] eT=313.4, eL2=256.7; hT=534.3, hL2=450.4
-- Step 1056, [T] Tloss=240.4, L2loss=187.5 (9.8s)
-- Step 1059, [V] eT=313.1, eL2=256.7; hT=535.2, hL2=451.6
------177/500------
-- 

# Training process (hard)

In [12]:
# direct to ckpt
ckpt_dir = 'ckpt/'+conf.tar_model+'_'+conf.loss_combination+'/'+str(conf.ef_dim)+'/'+conf.eye+'/'

train(conf, ckpt_dir, train_iter_, valid_iter_, False, True)

print('Finish All Training, 228')

INFO:tensorflow:Restoring parameters from pt_ckpt/flx_l2sc/12/L/dirl_48x64_example_L170-1020
Loading sucessfully
------0/500------
-- Step 0, [V*] eT=320.4, eL2=262.9; hT=539.0, hL2=453.7
-- Step 0, [T] Tloss=275.6, L2loss=218.4 (14.5s)
-- Step 102, [V] eT=358.7, eL2=288.2; hT=519.0, hL2=430.9
------1/500------
-- Step 204, [V] eT=331.7, eL2=277.7; hT=501.3, hL2=415.3
-- Step 204, [T] Tloss=280.3, L2loss=226.4 (9.7s)
-- Step 306, [V*] eT=282.7, eL2=245.3; hT=421.9, hL2=367.1
------2/500------
-- Step 408, [V*] eT=281.9, eL2=248.8; hT=411.2, hL2=360.8
-- Step 408, [T] Tloss=241.4, L2loss=203.7 (14.4s)
-- Step 510, [V*] eT=280.0, eL2=248.7; hT=406.0, hL2=357.6
------3/500------
-- Step 612, [V*] eT=276.2, eL2=246.2; hT=404.3, hL2=356.9
-- Step 612, [T] Tloss=236.7, L2loss=203.2 (14.3s)
-- Step 714, [V] eT=277.7, eL2=249.7; hT=399.1, hL2=353.6
------4/500------
-- Step 816, [V*] eT=274.2, eL2=245.8; hT=400.7, hL2=354.6
-- Step 816, [T] Tloss=239.5, L2loss=207.2 (14.5s)
-- Step 918, [V] eT

# Release Memory

In [13]:
sess.close()
print('Finish Training, 228\n')

Finish Training, 228

